In [ ]:
import numpy as np
import joblib
import pandas as pd

def engineer_features(patient_data):
    if isinstance(patient_data, dict):
        df = pd.DataFrame([patient_data])
    else:
        df = pd.DataFrame(patient_data, columns=[
            'age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg',
            'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal'
        ])

    df['age_over_55'] = (df['age'] >= 55).astype(int)
    df['asymptomatic_chest_pain'] = (df['cp'] == 4).astype(int)
    df['typical_angina'] = (df['cp'] == 1).astype(int)
    df['hypertension'] = (df['trestbps'] >= 140).astype(int)
    df['high_cholesterol'] = (df['chol'] > 240).astype(int)
    df['borderline_cholesterol'] = ((df['chol'] > 200) & (df['chol'] <= 240)).astype(int)
    df['low_max_hr'] = (df['thalach'] < 150).astype(int)
    df['exercise_angina_risk'] = df['exang']
    df['significant_st_depression'] = (df['oldpeak'] > 1.5).astype(int)
    df['flat_st_slope'] = (df['slope'] == 2).astype(int)
    df['downsloping_st'] = (df['slope'] == 3).astype(int)
    df['age_sex'] = df['age'] * df['sex']
    df['thalach_exang'] = df['thalach'] * (1 - df['exang'])
    df['combined_risk_score'] = (
        df['age_over_55'] +
        df['hypertension'] +
        df['high_cholesterol'] +
        df['fbs'] +
        df['exercise_angina_risk'] +
        df['significant_st_depression']
    )

    return df

def test_mi_prediction_model():
    print("Testing MI Prediction Model")
    print("--------------------------")

    try:
        print("Loading model from '/content/mi_complete_model.pkl'...")
        complete_model = joblib.load('/content/mi_complete_model.pkl')
        print("Model loaded successfully!")

        model = complete_model['model']
        preprocessor = complete_model['preprocessor']
        selector = complete_model['selector']
        feature_names = complete_model.get('feature_names', [])
    except FileNotFoundError:
        print("Error: Model file '/content/mi_complete_model.pkl' not found.")
        return
    except Exception as e:
        print(f"Error loading model: {e}")
        return

    sample_patient = {
        'age': 45,
        'sex': 1,
        'cp': 3,
        'trestbps': 120,
        'chol': 200,
        'fbs': 0,
        'restecg': 0,
        'thalach': 150,
        'exang': 0,
        'oldpeak': 1.2,
        'slope': 1,
        'ca': 0,
        'thal': 3
    }

    print("\nSample patient data:")
    for feature, value in sample_patient.items():
        print(f"  {feature}: {value}")

    try:

        patient_df = engineer_features(sample_patient)


        processed_data = preprocessor.transform(patient_df)


        selected_data = selector.transform(processed_data)


        prediction = model.predict(selected_data)[0]
        probability = model.predict_proba(selected_data)[0, 1]

        print("\nPrediction results:")
        print(f"  MI Risk: {'Yes' if prediction == 1 else 'No'}")
        print(f"  Probability of MI: {probability:.2%}")

        if hasattr(model, 'coef_') and len(feature_names) > 0:
            print("\nFeature contributions to this prediction:")
            coefficients = model.coef_[0]
            contrib = coefficients * selected_data[0]

            # Create feature contribution dictionary
            contributions = {}
            for i, feature in enumerate(feature_names):
                contributions[feature] = contrib[i]

            # Sort by absolute value
            sorted_contrib = sorted(contributions.items(), key=lambda x: abs(x[1]), reverse=True)

            for feature, contribution in sorted_contrib:
                print(f"  {feature}: {contribution:.4f} ({'increases' if contribution > 0 else 'decreases'} risk)")

    except Exception as e:
        print(f"Error during prediction: {e}")

    print("\nWould you like to test another patient? (y/n)")
    response = input()

    if response.lower() == 'y':
        print("\nEnter values for the patient:")
        new_patient = {}

        features = [
            'age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg',
            'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal'
        ]

        for feature in features:
            value = float(input(f"Enter value for {feature}: "))
            new_patient[feature] = value
        patient_df = engineer_features(new_patient)

        processed_data = preprocessor.transform(patient_df)

        selected_data = selector.transform(processed_data)

        prediction = model.predict(selected_data)[0]
        probability = model.predict_proba(selected_data)[0, 1]

        print("\nPrediction results:")
        print(f"  MI Risk: {'Yes' if prediction == 1 else 'No'}")
        print(f"  Probability of MI: {probability:.2%}")

if __name__ == "__main__":
    test_mi_prediction_model()

Testing MI Prediction Model
--------------------------
Loading model from '/content/mi_complete_model.pkl'...
Model loaded successfully!

Sample patient data:
  age: 45
  sex: 1
  cp: 3
  trestbps: 120
  chol: 200
  fbs: 0
  restecg: 0
  thalach: 150
  exang: 0
  oldpeak: 1.2
  slope: 1
  ca: 0
  thal: 3

Prediction results:
  MI Risk: No
  Probability of MI: 8.60%

Feature contributions to this prediction:
  ca_0.0: -1.5853 (decreases risk)
  combined_risk_score: -0.5638 (decreases risk)
  thal_3.0: -0.4479 (decreases risk)
  age_sex: 0.1683 (increases risk)
  thalach_exang: -0.0937 (decreases risk)
  oldpeak: 0.0524 (increases risk)
  thalach: -0.0005 (decreases risk)
  asymptomatic_chest_pain: 0.0000 (decreases risk)
  cp_4.0: 0.0000 (decreases risk)
  thal_7.0: 0.0000 (decreases risk)

Would you like to test another patient? (y/n)
y

Enter values for the patient:
Enter value for age: 68
Enter value for sex: 1
Enter value for cp: 4
Enter value for trestbps: 160
Enter value for chol: